In [1747]:
import numpy as np
import math
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import time
import networkx as nx
from cspy import BiDirectional
import os  
import matplotlib.pyplot as plt    
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import random
import csv

# Utility Functions

In [1749]:
# Function to calculate duration
def calculate_duration(departure, arrival):
    if pd.isna(arrival) or pd.isna(departure):
        return None  # Return None for missing values

    # Ensure inputs are strings
    departure = str(departure).strip()
    arrival = str(arrival).strip()

    next_day = False
    if "+1" in arrival:  # Handle next day flights
        arrival = arrival.replace("+1", "")
        next_day = True

    try:
        # Parse times
        departure_time = datetime.strptime(departure, "%H:%M")
        arrival_time = datetime.strptime(arrival, "%H:%M")
        if next_day:
            arrival_time += timedelta(days=1)
        
        # Calculate duration in minutes
        return (arrival_time - departure_time).seconds // 60
    except ValueError as e:
        print(f"Error parsing times: {departure}, {arrival}")
        return None


In [1750]:
def time_to_minutes(time_str):
    parts = time_str.split('+')  # Split on '+' to handle day increment
    hh, mm = map(int, parts[0].split(':'))  # Split HH:MM and convert to integers
    day_increment = int(parts[1]) * 24 * 60 if len(parts) > 1 else 0  # +1 means add 1440 minutes
    return hh * 60 + mm + day_increment

In [1751]:
import pandas as pd
# Function to convert time with day increment to HH:MM format
def convert_to_hhmm(time_str):
    # Split time and day increment
    if '+' in time_str:
        time_part, day_increment = time_str.split('+')
        return time_part
    else:
        return time_str



# Aircraft rotations and flights

In [1752]:
import pandas as pd
rotations = pd.read_csv('A01_6088570/rotations.csv', sep=" ", header=None, on_bad_lines='skip')
rotations = rotations.drop(columns=[3])
rotations.columns = ["FlightNumber", "Date", "Aircraft"]
rotations = rotations[:-1]
rotations["FlightNumber"] = rotations["FlightNumber"].astype(int)

flights = pd.read_csv('A01_6088570/flights.csv', sep=" ", header=None, on_bad_lines='skip')
flights.columns = ["FlightNumber", "Origin", "Destination", "DepartureTime", "ArrivalTime", "PreviousFlight"]
flights = flights[:-1]
flights["FlightNumber"] = flights["FlightNumber"].astype(int)

# Calculate the flight duration
flights["DurationMinutes"] = flights.apply(
    lambda row: calculate_duration(row["DepartureTime"], row["ArrivalTime"]), axis=1)
df_current_plan = pd.merge(flights, rotations, on="FlightNumber", how="left")



In [1753]:
# Apply the function to DepartureTime and ArrivalTime
df_current_plan['DepartureMinutes'] = df_current_plan['DepartureTime'].apply(time_to_minutes)
df_current_plan['ArrivalMinutes'] = df_current_plan['ArrivalTime'].apply(time_to_minutes)
df_current_plan['DepartureTime'] = df_current_plan['DepartureTime'].apply(convert_to_hhmm)
df_current_plan['ArrivalTime'] = df_current_plan['ArrivalTime'].apply(convert_to_hhmm)
df_current_plan['DepartureTime'] = pd.to_datetime(df_current_plan['DepartureTime'], format='%H:%M')
df_current_plan['DepartureTime'] = df_current_plan['DepartureTime'].dt.time
df_current_plan['ArrivalTime'] = pd.to_datetime(df_current_plan['ArrivalTime'], format='%H:%M')
df_current_plan['ArrivalTime'] = df_current_plan['ArrivalTime'].dt.time


In [1754]:
flights = df_current_plan['FlightNumber'].unique()
aircrafts = df_current_plan['Aircraft'].unique()
airports = set(df_current_plan['Origin'].unique()+df_current_plan['Destination'].unique())
print(len(flights),"flights between",len(airports),"airports operated with",len(aircrafts),"aircrafts")
duration_flights = df_current_plan.set_index('FlightNumber')['DurationMinutes'].to_dict()

608 flights between 35 airports operated with 85 aircrafts


In [1755]:
duration_flights = df_current_plan.set_index('FlightNumber')['DurationMinutes'].to_dict()

# Flight Delays

In [1756]:
df_delays = pd.read_csv('A01_6088570/alt_flights.csv', sep=" ", header=None, on_bad_lines='skip')
df_delays.columns = ['FlightNumber', 'Date', 'Delay']
df_delays = df_delays[:-1]
df_delays["FlightNumber"] = df_delays["FlightNumber"].astype(int)

In [1757]:
df_merged = pd.merge(df_current_plan, df_delays[['FlightNumber', 'Delay']], on='FlightNumber', how='left')
df_merged['Delay'] = df_merged['Delay'].fillna(0)
df_merged['Delay'] = df_merged['Delay'].astype(int)
df_merged = df_merged[df_merged['Delay'] != -1]
df_merged['DepartureMinutes'] += df_merged['Delay']
df_merged['ArrivalMinutes'] += df_merged['Delay']

In [1758]:
df_merged['NewDepartureTime'] = df_merged['DepartureMinutes'].apply(lambda x: x - 1440 if x >= 1440 else x)
df_merged['NewArrivalTime'] = df_merged['ArrivalMinutes'].apply(lambda x: x - 1440 if x >= 1440 else x)
df_merged['NewDepartureTime'] = pd.to_datetime(df_merged['NewDepartureTime'], unit='m').dt.strftime('%H:%M')
df_merged['NewDepartureTime'] = pd.to_datetime(df_merged['NewDepartureTime'], format='%H:%M')
df_merged['NewDepartureTime'] = df_merged['NewDepartureTime'].dt.time
df_merged['NewArrivalTime'] = pd.to_datetime(df_merged['NewArrivalTime'], unit='m').dt.strftime('%H:%M')
df_merged['NewArrivalTime'] = pd.to_datetime(df_merged['NewArrivalTime'], format='%H:%M')
df_merged['NewArrivalTime'] = df_merged['NewArrivalTime'].dt.time

In [1759]:
df_current_plan = df_merged.copy()

# Flights origin and destination and aircraft positions

In [1760]:
flight_origin = df_current_plan.set_index('FlightNumber')['Origin'].to_dict()
flight_dest = df_current_plan.set_index('FlightNumber')['Destination'].to_dict()
flight_start_time = df_current_plan.set_index('FlightNumber')['DepartureMinutes'].to_dict()
flight_end_time = df_current_plan.set_index('FlightNumber')['ArrivalMinutes'].to_dict()
flight_start_time_hrs = df_current_plan.set_index('FlightNumber')['NewDepartureTime'].to_dict()
flight_end_time_hrs = df_current_plan.set_index('FlightNumber')['NewArrivalTime'].to_dict() 

In [1761]:
merged_df = df_current_plan.sort_values(by="NewDepartureTime").reset_index(drop=True)
start_positions = merged_df.groupby("Aircraft").first().reset_index()  # First flight
end_positions = merged_df.groupby("Aircraft").last().reset_index() 
aircrafts_startpositions_airc = start_positions.set_index('Aircraft')['Origin'].to_dict()
aircrafts_endpositions_airc = end_positions.set_index('Aircraft')['Destination'].to_dict()


# Passengers itinerary

In [1762]:
import pandas as pd
with open('A01_6088570/itineraries.csv', 'r') as file:
    data = []
    # Read the file line by line
    for line in file:
        # parts = [part.strip() for part in line.split(" ") if part.strip()]
        if line.startswith("#") :
            continue
        parts = line.strip().split(" ")
        data.append(parts)

# Create a list to store the rows to create the dataframe later
expanded_data = []

# Iterate over each row in the data
for row in data:
    # Extract relevant information
    itinerary_number = row[0]
    type_of_itinerary = row[1]
    cost = float(row[2])
    num_passengers = int(row[3])
    flight_details = row[4:]

    # Handle single-flight itinerary (no additional flight details)
    if len(flight_details) == 3:
        expanded_data.append({
            'ItineraryNumber': itinerary_number,
            'Type': type_of_itinerary,
            'Cost': cost,
            'NumPassengers': num_passengers,
            'FlightNumber': int(flight_details[0]),
            'Date': flight_details[1],
            'CabinClass': flight_details[2]
        })
    else:
        # Multi-flight itinerary
        for i in range(0, len(flight_details), 3):
            flight_number = int(flight_details[i])
            date = flight_details[i+1]
            cabin_class = flight_details[i+2]

            expanded_data.append({
                'ItineraryNumber': itinerary_number,
                'Type': type_of_itinerary,
                'Cost': cost,
                'NumPassengers': num_passengers,
                'FlightNumber': flight_number,
                'Date': date,
                'CabinClass': cabin_class
            })

# Create the DataFrame
df_itineraries = pd.DataFrame(expanded_data)

In [1763]:
df_itineraries['total_cost'] = df_itineraries['Cost']*df_itineraries['NumPassengers']
flight_revenue = df_itineraries.groupby(['FlightNumber'])['total_cost'].agg('sum').to_dict() 
flight_n_pass = df_itineraries.groupby(['FlightNumber'])['NumPassengers'].agg('sum').to_dict() 

# Aircrafts costs and seating configuration

In [1764]:
df_aircrafts = pd.read_csv('A01_6088570/aircraft.csv', sep=" ", header=None, on_bad_lines='skip')
df_aircrafts = df_aircrafts[:-1]
df_aircrafts = df_aircrafts.drop(df_aircrafts.columns[9:], axis=1)
df_aircrafts.columns = ['Aircraft', 'Model', 'Family', "Config", "Dist", "Cost/h", "TurnRound", "Transit", "Orig"]
df_aircrafts[['F', 'B', 'E']] = df_aircrafts['Config'].str.split('/', expand=True)
df_aircrafts[['F', 'B', 'E']] = df_aircrafts[['F', 'B', 'E']].apply(pd.to_numeric, errors='coerce').replace(-1, 999999)
df_aircrafts['Cost_per_minute'] = df_aircrafts['Cost/h']/60
aircrafts_cost = df_aircrafts.set_index('Aircraft')['Cost_per_minute'].to_dict()
aircrafts_turnround = df_aircrafts.set_index('Aircraft')['TurnRound'].to_dict()

# Aircraft compatible arcs

In [1765]:
aircraft_flights = df_current_plan.groupby(['Aircraft']).apply(lambda x: x['FlightNumber'].tolist()).to_dict()
flight_arcs_for_each_aircraft = {}
deltaplus_flightarcs = {}
deltaminus_flightarcs = {}

In [1766]:
aircraft_flights = df_current_plan.groupby(['Aircraft']).apply(lambda x: x['FlightNumber'].tolist()).to_dict()
flight_arcs_for_each_aircraft = {}
deltaplus_flightarcs = {}
deltaminus_flightarcs = {}
for a in aircraft_flights:
    aircraft_flights[a] += ['source_%s'%a,'sink_%s'%a]
    flight_origin['source_%s'%a] = aircrafts_endpositions_airc[a]
    flight_dest['source_%s'%a] = aircrafts_startpositions_airc[a]
    flight_origin['sink_%s'%a] = aircrafts_endpositions_airc[a]
    flight_dest['sink_%s'%a] = aircrafts_startpositions_airc[a]

    flight_start_time['source_%s'%a] = -max(aircrafts_turnround.values())
    flight_end_time['source_%s'%a] = -max(aircrafts_turnround.values())
    flight_start_time_hrs['source_%s'%a] = datetime.strptime('0:0', '%H:%M').time()
    flight_end_time_hrs['source_%s'%a] = datetime.strptime('0:0', '%H:%M').time()

    flight_start_time['sink_%s'%a] = max(df_current_plan['ArrivalMinutes']) + max(aircrafts_turnround.values())
    flight_end_time['sink_%s'%a] = max(df_current_plan['ArrivalMinutes']) + max(aircrafts_turnround.values())
    flight_start_time_hrs['sink_%s'%a] = datetime.strptime('23:59', '%H:%M').time()
    flight_end_time_hrs['sink_%s'%a] = datetime.strptime('23:59', '%H:%M').time()

    flight_arcs_for_each_aircraft[a] = []
    deltaplus_flightarcs[a] = {f: [] for f in aircraft_flights[a]}
    deltaminus_flightarcs[a] = {f: [] for f in aircraft_flights[a]}

    for f1 in aircraft_flights[a]:
        for f2 in aircraft_flights[a]:
            if f1!=f2 and flight_end_time[f1] + aircrafts_turnround[a] < flight_start_time[f2] and flight_dest[f1] == flight_origin[f2]: 
                # print(f1, f2, flight_end_time[f1], flight_start_time[f2],aircrafts_turnround[a], flight_dest[f1], flight_origin[f2])
                flight_arcs_for_each_aircraft[a].append((f1,f2))
                deltaplus_flightarcs[a][f1].append(f2)
                deltaminus_flightarcs[a][f2].append(f1) 
            # allow to connect source and target directly for the case that aircraft is not used at all    
            elif str(f1).startswith('source') and str(f2).startswith('sink'):
                flight_arcs_for_each_aircraft[a].append((f1,f2))
                deltaplus_flightarcs[a][f1].append(f2)
                deltaminus_flightarcs[a][f2].append(f1)

# Airport disruptions

In [1767]:
import pandas as pd

# Input data as a string
with open('A01_6088570/airports.csv', 'r') as file:
    data = []
    # Read the file line by line
    for line in file:
        # parts = [part.strip() for part in line.split(" ") if part.strip()]
        if line.startswith("#") :
            continue
        parts = line.strip().split(" ")
        data.append(parts)

# Parse the data
rows = []
for row in data:
    airport = row[0]
    details = row[1:]
    for i in range(0, len(details), 4):
        max_departures = int(details[i])
        max_arrivals = int(details[i+1])
        time_start = details[i+2]
        # convert time start to HH:MM and then .dt.time to get the time format
        time_start = pd.to_datetime(time_start, format='%H:%M').time()
        time_end = details[i+3]
        # convert time end to HH:MM and then .dt.time to get the time format
        time_end = pd.to_datetime(time_end, format='%H:%M').time()
        rows.append((airport, max_departures, max_arrivals, time_start, time_end))
        
# Create a DataFrame
df_airport = pd.DataFrame(rows, columns=["Airport", "MaxDepartures", "MaxArrivals", "StartTime", "EndTime"])
    


# Model

In [1768]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("airline_disruption")

In [1769]:
x, y = {}, {}
for a in aircrafts:
    for f in aircraft_flights[a]:
        x[a,f] = model.addVar(name="x_%s,%s"%(a,f), vtype=GRB.BINARY)

    for (f1,f2) in flight_arcs_for_each_aircraft[a]:
        y[a,f1,f2] = model.addVar(name="y_%s,%s,%s"%(a,f1,f2), vtype=GRB.BINARY)
model.update()

# Objective

In [1770]:
objective = gp.quicksum((1-x[a,f])*flight_revenue[f] for a in aircrafts for f in aircraft_flights[a] if f in flight_revenue) # Cancellation cost
objective += gp.quicksum(x[a,f]*duration_flights[f]*aircrafts_cost[a] for a in aircrafts for f in aircraft_flights[a] if f in duration_flights) # Operating cost
model.setObjective(objective, sense=GRB.MINIMIZE)

# Constraint 1 and 2: FLow conservation

In [1771]:
for a in aircrafts:
    model.addConstr(sum(y[a,'source_%s'%a,f2] for f2 in deltaplus_flightarcs[a]['source_%s'%a]) == 1)
    model.addConstr(sum(y[a,f1,'sink_%s'%a] for f1 in deltaminus_flightarcs[a]['sink_%s'%a]) == 1)
    for f in aircraft_flights[a]: 
        if str(f)[0] != 's':
            model.addConstr(sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f]) == sum(y[a,f1,f] for f1 in deltaminus_flightarcs[a][f]))
for a in aircrafts:
    for f in aircraft_flights[a]:
        model.addConstr(x[a,f] <= sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f])) # flight f is chosen only if it is traversed

# Airport disruption Constraint

In [1772]:
# Ensure times are datetime objects for comparison
df_airport['StartTime'] = pd.to_datetime(df_airport['StartTime'], format='%H:%M:%S').dt.time
df_airport['EndTime'] = pd.to_datetime(df_airport['EndTime'], format='%H:%M:%S').dt.time

# Iterate over airports and their disruption intervals
for airport in df_airport['Airport'].unique():
    airport_disruptions = df_airport[df_airport['Airport'] == airport]
    
    for _, row in airport_disruptions.iterrows():
        start_time = row['StartTime']
        end_time = row['EndTime']
        max_departures = row['MaxDepartures']
        max_arrivals = row['MaxArrivals']
        
        # Get flights departing or arriving at this airport in the given interval
        flights_departing = [f for a in aircrafts for f in aircraft_flights[a]
                             if (flight_origin[f] == airport) and (start_time <= flight_start_time_hrs[f]) and (flight_start_time_hrs[f] < end_time) and (str(f).startswith('source') == False) and (str(f).startswith('sink') == False)]
        # print(flights_departing,start_time,end_time,max_departures,airport)
        
        flights_arriving = [f for a in aircrafts for f in aircraft_flights[a]
                            if (flight_dest[f] == airport) and (start_time <= flight_end_time_hrs[f]) and (flight_end_time_hrs[f] < end_time) and (str(f).startswith('source') == False) and (str(f).startswith('sink') == False)]
        # print(flights_arriving,start_time,end_time,max_arrivals,airport)
        
        # Add constraints for departures and arrivals
        if len(flights_departing) > 0:
            model.addConstr(
                sum(x[a, f] for a in aircrafts for f in flights_departing if f in aircraft_flights[a]) <= max_departures,
                name=f"{airport}_max_departures_{start_time}_{end_time}")
        if len(flights_arriving) > 0:
            model.addConstr(
                sum(x[a, f] for a in aircrafts for f in flights_arriving if f in aircraft_flights[a]) <= max_arrivals,
                name=f"{airport}_max_arrivals_{start_time}_{end_time}")
            
model.write("airline_disruption.lp")
model.optimize()
operated_flights = {a: [f for f in aircraft_flights[a] if x[a,f].X > .5 if str(f)[0] != 's'] for a in aircrafts}

print("\nNet Cost: Euros",round(model.objVal/10**6,2),'million')
print("Optimal number of flights served:",sum(len(operated_flights[a]) for a in aircrafts))
print("Optimal number of passengers transported:",sum(sum(flight_n_pass[f] for f in aircraft_flights[a] if x[a,f].X > .5) for a in aircrafts))
print("Optimal number of aircrafts utilized:",sum([1 if len(operated_flights[a]) > 0 else 0 for a in aircrafts]))



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Academic license 2544493 - for non-commercial use only - registered to va___@iisc.ac.in
Optimize a model with 1733 rows, 3142 columns and 8980 nonzeros
Model fingerprint: 0x9ce885c7
Variable types: 0 continuous, 3142 integer (3142 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+02, 7e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 1.074404e+07
Presolve removed 1507 rows and 2703 columns
Presolve time: 0.04s
Presolved: 226 rows, 439 columns, 969 nonzeros
Found heuristic solution: objective 4878655.9000
Variable types: 0 continuous, 439 integer (438 binary)
Found heuristic solution: objective 4801564.4667

Root relaxation: objective 4.206718e+06,

# Cancellation, downgradations and delay costs

In [1773]:
with open('A01_6088570/config.csv', 'r') as file:
    data = []
    # Read the file line by line
    for line in file:
        # parts = [part.strip() for part in line.split(" ") if part.strip()]
        if line.startswith("#") :
            continue
        parts = line.strip().split(" ")
        data.append(parts)
delay_dict = {}
cancellation_dict = {}
downgrade_dict = {}
i = 0
for row in data:
    if i ==1:
        delay_dict[row[0]] = row[2] 
        delay_dict[row[9]] = row[11]
        delay_dict[row[18]] = row[20]
    if i == 2:
        cancellation_dict[row[0]] = row[2]
        cancellation_dict[row[9]] = row[11]
        cancellation_dict[row[18]] = row[20]
    if i == 4:
        downgrade_dict[(row[0],row[1])] = row[3]
        downgrade_dict[(row[12],row[13])] = row[15]
        downgrade_dict[(row[24],row[25])] = row[27]
    if i == 6:
        alpha_delay = row[0]
        alpha_cancellation = row[1]
        alpha_downgrade = row[2]
    i += 1

# Passengers

In [1774]:
operated_flights = {a: [f for f in aircraft_flights[a] if x[a,f].X > .5 if str(f)[0] != 's'] for a in aircrafts}
# flight lists in operated flights
flights_list = []
for a in aircrafts:
    for f in operated_flights[a]:
        flights_list.append(f)
# start of iternaary and end of itinerary
start_itinerary = {}
end_itinerary = {}
for i in df_itineraries['ItineraryNumber'].unique():
    start_itinerary[i] = df_itineraries[df_itineraries['ItineraryNumber'] == i]['FlightNumber'].iloc[0]
    end_itinerary[i] = df_itineraries[df_itineraries['ItineraryNumber'] == i]['FlightNumber'].iloc[-1]
    
# start time of itinerary and end time of itinerary
start_time_itinerary = {}
end_time_itinerary = {}
for i in df_itineraries['ItineraryNumber'].unique():
    start_time_itinerary[i] = flight_start_time[start_itinerary[i]]
    end_time_itinerary[i] = flight_end_time[end_itinerary[i]]
aircraft_seats = df_aircrafts.set_index('Aircraft')[['F', 'B', 'E']].T.to_dict()

start_itinerary_class = {}
end_itinerary_class = {}
for i in df_itineraries['ItineraryNumber'].unique():
    start_itinerary_class[i] = df_itineraries[df_itineraries['ItineraryNumber'] == i]['CabinClass'].iloc[0]
    end_itinerary_class[i] = df_itineraries[df_itineraries['ItineraryNumber'] == i]['CabinClass'].iloc[-1]
num_passengers = df_itineraries.set_index('ItineraryNumber')['NumPassengers'].to_dict()
flight_aircraft = df_current_plan.set_index('FlightNumber')['Aircraft'].to_dict()

In [1775]:
graphs = {}
adj_lists = {}
for i in start_itinerary:
    # Initialize adjacency lists for incoming and outgoing edges
    outgoing_adjacency = {}
    incoming_adjacency = {}
    arcs = []
    
    # Extract relevant details for the current itinerary
    start_flight = start_itinerary[i]
    end_flight = end_itinerary[i]
    start_airport = flight_origin[start_flight]
    end_airport = flight_dest[end_flight]
    start_time = start_time_itinerary[i]
    end_time = end_time_itinerary[i]
    starting_class = start_itinerary_class[i]
    ending_class = end_itinerary_class[i]
    num_passengers_itinerary = num_passengers[i]
    cancellation_cost_source = float(cancellation_dict[starting_class])
    cancellation_cost_sink = float(cancellation_dict[ending_class])
    
    source_node = ("Source", starting_class, i, 0)
    sink_node = ("Sink", ending_class, i, 0)
    
    # Candidate nodes
    start_candidates = []
    end_candidates = []
    rest_candidates = []
    direct_candidates = []

    for flight in flight_start_time_hrs:
        if flight_start_time[flight] >= start_time and flight_end_time[flight] >= end_time:
            if flight in flights_list:
                aircraft = flight_aircraft[flight]
                for cls in ['F', 'B', 'E']:
                    if num_passengers_itinerary <= aircraft_seats[aircraft][cls]:
                        downgrade_cost = float(downgrade_dict.get((starting_class, cls), 0)) * num_passengers_itinerary
                        delay_cost = (flight_end_time[flight] - end_time) * float(delay_dict[cls]) * num_passengers_itinerary if flight_end_time[flight] > end_time else 0
                        total_cost = delay_cost + downgrade_cost
                        
                        if flight_origin[flight] == start_airport and flight_dest[flight] == end_airport:
                            direct_candidates.append((flight, cls, i, total_cost))
                        elif flight_origin[flight] == start_airport:
                            start_candidates.append((flight, cls, i, total_cost))
                        elif flight_dest[flight] == end_airport:
                            end_candidates.append((flight, cls, i, total_cost))
                        else:
                            rest_candidates.append((flight, cls, i, total_cost))    
    
    arcs.append([source_node, sink_node, max(cancellation_cost_sink, cancellation_cost_source) * num_passengers_itinerary])

    # Connect SOURCE to start_candidates
    arcs.extend([[source_node, start_node, start_node[3]] for start_node in start_candidates])

    # Connect end_candidates to SINK
    arcs.extend([[end_node, sink_node, sink_node[3]] for end_node in end_candidates])

    # Connect SOURCE directly to direct_candidates
    arcs.extend([[source_node, direct_node, direct_node[3]] for direct_node in direct_candidates])

    # Connect direct_candidates directly to SINK
    arcs.extend([[direct_node, sink_node, sink_node[3]] for direct_node in direct_candidates])

    # Create arcs with time compatibility
    arcs.extend([
        [start_node, middle_node, middle_node[3]] 
        for start_node in start_candidates 
        for middle_node in rest_candidates
        if flight_dest[start_node[0]] == flight_origin[middle_node[0]] and flight_end_time[start_node[0]] < flight_start_time[middle_node[0]]
    ])

    arcs.extend([
        [middle_node, end_node, end_node[3]]
        for middle_node in rest_candidates 
        for end_node in end_candidates
        if flight_dest[middle_node[0]] == flight_origin[end_node[0]] and flight_end_time[middle_node[0]] < flight_start_time[end_node[0]]
    ])

    # Connect start_candidates directly to end_candidates
    arcs.extend([
        [start_node, end_node, end_node[3]] 
        for start_node in start_candidates 
        for end_node in end_candidates
        if flight_dest[start_node[0]] == flight_origin[end_node[0]]
    ])
    graphs[i] = arcs
    # Build adjacency lists
    for arc in arcs:
        source, destination, cost = arc
        
        # Populate outgoing adjacency list
        if source not in outgoing_adjacency:
            outgoing_adjacency[source] = []
        outgoing_adjacency[source].append((destination, cost))
        
        # Populate incoming adjacency list
        if destination not in incoming_adjacency:
            incoming_adjacency[destination] = []
        incoming_adjacency[destination].append((source, cost))
    
    # Store both adjacency lists for this graph
    adj_lists[i] = {
        "outgoing": outgoing_adjacency,
        "incoming": incoming_adjacency
    }

# Initialize model

In [1776]:
passenger_model = gp.Model("passenger_allocation")
passenger_model.Params.OutputFlag = 0
y = {}
all_variables = []  # Track all variables added so far
active_constraints = []  # Track active constraints for removal

# Initialize an expression for the objective
objective = gp.LinExpr()

for graph, adj_list in adj_lists.items():
    outgoing_adjacency = adj_list["outgoing"]
    
    # Process only outgoing edges
    for source, neighbors in outgoing_adjacency.items():
        if source[0] == "Source":
            for destination, cost in neighbors:
                if destination[0] == "Sink":
                    var_name = (source[0], destination[0], destination[1], destination[2])
                    all_variables.append((source, destination))
                        # Add decision variable if not already added
                    # if var_name not in all_variables:
                    y[var_name] = passenger_model.addVar(
                        name="y_%s,%s,%s,%s" % (var_name[0], var_name[1], var_name[2], var_name[3]),
                        vtype=GRB.CONTINUOUS,
                        lb=0
                    )
                        # all_variables[var_name] = y[var_name]

                        # Update objective function
                    objective += y[var_name] * cost

                    # Add demand constraint for new variables
                    constraint_name = f"demand_{destination[2]}"
                    active_constraints.append(constraint_name)
                    passenger_model.addConstr(
                        y[var_name] == num_passengers[destination[2]],
                        name=constraint_name
                    )

# Set the accumulated objective function
passenger_model.setObjective(objective, GRB.MINIMIZE)



In [1777]:
passenger_model.optimize()
initial_cost = passenger_model.objVal
duals = {}
for constr in passenger_model.getConstrs():
    duals[constr.ConstrName] = constr.Pi

# Shortest paths

In [1778]:
def find_shortest_paths(graphs, duals):
    """
    Finds shortest paths using the cspy library for each graph in the system.
    """
    shortest_paths = {}
    for graph, arcs in graphs.items():
        if len(arcs) == 1:
            continue

        G = nx.DiGraph(directed=True, n_res=1)
        for arc in arcs:
            source, destination, cost = arc[0], arc[1], float(arc[2])
            reduced_cost = cost

            if source[0] == 'Source':
                G.add_edge('Source', source, res_cost=[1], weight=0)
                reduced_cost -= duals.get(f"demand_{int(graph)}", 0)

            if destination[0] == 'Sink':
                G.add_edge(destination, 'Sink', res_cost=[1], weight=0)

            reduced_cost -= duals.get(f'flow_conservation_{destination[0]}_{int(graph)}', 0)
            reduced_cost += duals.get(f'flow_conservation_{source[0]}_{int(graph)}', 0)
            reduced_cost -= duals.get(f'capacity_{source[0]}_{destination[0]}_{destination[1]}', 0)

            G.add_edge(source, destination, res_cost=[1], weight=reduced_cost)

        bidirectional = BiDirectional(G, max_res=[len(arcs) + 1], min_res=[0], direction='both', elementary=True)
        bidirectional.run()
        if bidirectional.total_cost < -1e-6:
            shortest_paths[graph] = bidirectional.path[1:-1]
    
    return shortest_paths

In [1779]:
shortest_paths = find_shortest_paths(graphs, duals)

No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true

# Function to add arcs from the shortest paths


In [1780]:
def add_arcs_from_graph(shortest_paths, adj_lists, passenger_model, y, all_variables, objective):
    """
    Adds arcs from the shortest paths to the optimization model.
    """
    for itinerary_id, path in shortest_paths.items():
        for i in range(len(path) - 1):
            source = path[i]
            destination = path[i + 1]

            # Find the cost of the edge from the graph
            edge_cost = None
            for edge in adj_lists[itinerary_id]['outgoing'][source]:
                if edge[0] == destination:
                    edge_cost = edge[1]
                    break
            
            if edge_cost is None:
                raise ValueError(f"Edge cost not found for {source} -> {destination} in graph {itinerary_id}.")

            var_name = (source[0], destination[0], destination[1], destination[2])
            all_variables.append((source, destination))

            # Add variable to the model
            y[var_name] = passenger_model.addVar(
                name="y_%s,%s,%s,%s" % (var_name[0], var_name[1], var_name[2], var_name[3]),
                vtype=GRB.CONTINUOUS,
                lb=0
            )
            objective += y[var_name] * edge_cost

    passenger_model.setObjective(objective, GRB.MINIMIZE)
    for constr in passenger_model.getConstrs():
        passenger_model.remove(constr)
    passenger_model.update()


#  Function to add constraints

In [1781]:

def add_constraints(passenger_model, y, all_variables, num_passengers, flight_aircraft, aircraft_seats):
    """
    Adds demand, flow conservation, and capacity constraints to the model.
    """
    # 3.1 Demand Constraints
    variables_by_itinerary = {}
    for source, destination in all_variables:
        if source[0] == "Source":
            itinerary_number = source[2]
            var_name = (source[0], destination[0], destination[1], destination[2])
            variables_by_itinerary.setdefault(itinerary_number, []).append(var_name)

    for itinerary_number, var_list in variables_by_itinerary.items():
        passenger_model.addConstr(
            sum(y[var_name] for var_name in var_list) == num_passengers[itinerary_number],
            name=f"demand_{itinerary_number}"
        )

    # 3.2 Flow Conservation Constraints
    variables_by_itinerary = {}
    for source, destination in all_variables:
        itinerary_number = source[2] # Extract itinerary number
        var_name = (source[0], destination[0], destination[1], destination[2])
        if itinerary_number not in variables_by_itinerary:
            variables_by_itinerary[itinerary_number] = []
        variables_by_itinerary[itinerary_number].append(var_name)
    for itinerary_number, var_list in variables_by_itinerary.items():
        for arcs in var_list:
            if arcs[0] == 'Source':
                continue
            
            source = arcs[0]
            outgoing_flow = sum(set(y[var_name] for var_name in var_list if var_name[0] == source))
            incoming_flow = sum(set(y[var_name] for var_name in var_list if var_name[1] == source))
            passenger_model.addConstr(incoming_flow - outgoing_flow == 0, name=f"flow_conservation_{source}_{itinerary_number}")

    # 3.3 Capacity Constraints
    variables_by_group = {}
    for source, destination in all_variables:
        origin, destination_node, class_name = source[0], destination[0], destination[1]
        group_key = (origin, destination_node, class_name)
        variables_by_group.setdefault(group_key, []).append((origin, destination_node, class_name, source[2]))

    for source_destination_key, var_list in variables_by_group.items():
        source, destination, cabin_type = source_destination_key
        if destination == 'Sink':
            continue
        aircraft = flight_aircraft[destination]
        passenger_model.addConstr(
            sum(y[var_name] for var_name in var_list) <= aircraft_seats[aircraft][cabin_type],
            name=f"capacity_{source}_{destination}_{cabin_type}"
        )

    passenger_model.update()

# Main Column Generation Loop

In [1782]:
def column_generation(graphs, duals, adj_lists, passenger_model, y, all_variables, num_passengers, flight_aircraft, aircraft_seats):
    """
    Runs the column generation algorithm iteratively.
    """
    i = 0
    while True:
        # Step 4.1: Solve the subproblem to find shortest paths
        shortest_paths = find_shortest_paths(graphs, duals)

        # Step 4.2: Add new columns (variables) to the master problem
        objective = passenger_model.getObjective()
        add_arcs_from_graph(shortest_paths, adj_lists, passenger_model, y, all_variables, objective)

        # Step 4.3: Add constraints to the model
        add_constraints(passenger_model, y, all_variables, num_passengers, flight_aircraft, aircraft_seats)

        # Step 4.4: Optimize the master problem
        passenger_model.optimize()

        # Step 4.5: Get dual values and check termination criteria
        duals = {constr.ConstrName: constr.Pi for constr in passenger_model.getConstrs()}
        if i == 5 or not shortest_paths: # Implement termination check
            break

In [1783]:
column_generation(graphs, duals, adj_lists, passenger_model, y, all_variables, num_passengers, flight_aircraft, aircraft_seats)

No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true.
Consider setting elementary to false.
No negative cost cycle has been found and elementary set to true

In [1784]:
passenger_model.write("passenger_allocation.lp")

In [1785]:
for var in passenger_model.getVars():
    var.Vtype = GRB.INTEGER
    
passenger_model.optimize()

final_cost = passenger_model.objVal

In [1786]:
print("\nNet Cost: Euros",round(final_cost/10**6,2),'million')


Net Cost: Euros 46.88 million
